# One Hot Encoder

## Introdução

Este notebook está na mesma categoria que o "LabelEncoder.ipynb" pois ambos versam sobre a codificação de categorias. No caso do label encoder, a gente transformava strings em números de modo que havia uma relação de ordem entre os elementos. Por exemplo, se temos strings "cartão gold" e "cartão black", é mais adequado atribuirmos um número x para o black e y para o gold de modo que x > y pois é bem sabido que um cartão de crédito black é mais valioso que um gold.

Contudo, ocorre ocasiões nas quais não há sentido em estabelecer uma relação de ordem entre as variáveis. Por exemplo, por que a cor "violeta" valeria menos ou mais do que a cor "azul"? Para sanar esse tipo de problema, usamos o *One Hot Encoding*, que transforma essas strings em vetores, e assim conseguimos transformar variáveis categóricas em numéricas de modo que o algoritmo de machine learning não tire conclusões errôneas.

## Dados Iniciais

Criaremos em essência o mesmo conjunto de dados, e a partir dele faremos um array e um DataFrame para mostrar que em ambas as tipagens o código funciona.

In [11]:
from numpy import array
from pandas import DataFrame

lista = ['vermelho', 'azul', 'rosa', 'azul', 'bege', 'vermelho']

dicionario = {'cores': lista}
df = DataFrame(data = dicionario)
df

,cores
0,vermelho
1,azul
2,rosa
3,azul
4,bege
5,vermelho


In [16]:
x = array(lista)
x

array(['vermelho', 'azul', 'rosa', 'azul', 'bege', 'vermelho'],
      dtype='<U8')

## ```OneHotEncoder```

Para fazermos essa codificação de categorias, importamos a classe ```OneHotEncoder``` de ```sklearn.preprocessing``` . Daí, é só criar uma instância, e colocar no parâmetro ```sparse``` como ```False``` para indicar que queremos um array - ao invés de matriz - depois de efetuar a codificação.

In [20]:
from sklearn.preprocessing import OneHotEncoder

codificador = OneHotEncoder(sparse = False)

Para fazermos a codificação de fato, é só usar o método ```fit_transform```, e colocar na entrada os dados a serem codificados. 

No caso de um ```DataFrame```, é só colocar diretamente no argumento do método.

In [19]:
u = codificador.fit_transform(df)
u

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]])

Já no caso de ```array```, precisamos redimensionar os dados, pois os dados não possuem uma coluna, como podemos ver abaixo.

In [23]:
x.shape

(6,)

São apenas 6 linhas e "zero" colunas. Para sanar sanar esse problema, é só fazer

In [26]:
xx = x.reshape(len(x),1)

E agora sim procedemos da mesma maneira que no caso do ```DataFrame```.

In [27]:
v = codificador.fit_transform(xx)
v

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]])